In [1]:
import utils
import math
import re
import itertools
import matplotlib.pyplot as plt
from collections import namedtuple

## Day 4: Passport Processing 

[#](https://adventofcode.com/2020/day/4). We have password data in the form:

code | name | comment
---  | ---- | -----
byr | (Birth Year) |
iyr | (Issue Year) | 
eyr | (Expiration Year) | 
hgt | (Height) | 
hcl | (Hair Color) | 
ecl | (Eye Color) | 
pid | (Passport ID) |
cid | (Country ID) | ignored

For part 1, only the **cid** field can be missing, the other seven have to be present for the data to be valid.

This is interesting becuase we basically have to write a data parser. So once again I'll use a named tuple, though a data class might be better as that has built in validation features.

In [3]:
test4 = """ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in"""

inp4 = utils.get_input(4, splitlines=False)

In [4]:
passport_fields = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid", "cid"]
# the defaults apply from the left
Passport = namedtuple("Passport", passport_fields, defaults=[""])

In [5]:
inp = inp4
passports = []

for line in inp.split("\n\n"):
    data = ["" for i in range(8)]
    line = line.replace("\n", " ").split(" ")
    #print(line)
    
    for field in line:
        key, val = field.split(":")
        
        idx = passport_fields.index(key)
        data[idx] = val
    #print(data)
    p = Passport(*data)
    passports.append(p)
    
passports[:5]

[Passport(byr='2027', iyr='1928', eyr='2039', hgt='190', hcl='a5ac0f', ecl='#25f8d2', pid='476113241', cid='150'),
 Passport(byr='1929', iyr='2013', eyr='2026', hgt='168cm', hcl='#fffffd', ecl='hzl', pid='920076943', cid='169'),
 Passport(byr='1948', iyr='2011', eyr='2023', hgt='156cm', hcl='#6b5442', ecl='brn', pid='328412891', cid=''),
 Passport(byr='1950', iyr='2019', eyr='2020', hgt='189cm', hcl='#602927', ecl='amb', pid='674907993', cid='279'),
 Passport(byr='1976', iyr='2015', eyr='2022', hgt='178cm', hcl='#341e13', ecl='hzl', pid='473630095', cid='')]

So now to check that the first seven fields are present. First to double check that an empty string evaluates to false:

In [10]:
all(["test", ""])

False

In [6]:
sum([all(p[:7])for p in passports])

190

## Part 2

Now we need to validate data ranges - the table from above becomes:. 

code | name | range
---  | ---- | -----
byr | (Birth Year) | 4 digits, 1920-2020
iyr | (Issue Year) | 4 digits, 2010-2020
eyr | (Expiration Year) | 4 digits, 2020-2030
hgt | (Height) | num, cm 150-193, in 59-63
hcl | (Hair Color) | # followed by six chars 0-9 or a-f
ecl | (Eye Color) | one of amb blu brn gry grn hzl oth
pid | (Passport ID) | nine digit num including leading zeros
cid | (Country ID) | ignored

We already have a list of passports so I'll reuse that.

First up writing some helper functions to validate specific passport fields:

In [11]:
def valid_height(txt):
    num, system = txt[:-2], txt[-2:]
    try:
        num = float(num)
    except:
        return False
    if system == "cm":
        return 150 <= num <= 193
    elif system == "in":
        return 59 <= num <= 76
    else:
        return False
    
valid_height(p.hgt)

False

In [12]:
def valid_range(txt, lo=1920, hi=2002, num_digits=4):
    if len(txt) != num_digits:
        return False
    return lo <= int(txt) <= hi

valid_range(p.byr)

True

In [13]:
valid_eyes = "amb blu brn gry grn hzl oth".split(" ")
valid_eyes

['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

In [14]:
def valid_hair(txt):
    if re.match(r"^#[0-9a-f]{6}$", txt):
        return True
    else:
        return False
    
valid_hair("#abc123")

True

In [15]:
def valid_passport(p):
    return (valid_range(p.byr) &
            valid_range(p.iyr, 2010, 2020) & 
            valid_range(p.eyr, 2020, 2030) & 
            valid_height(p.hgt) &
            valid_hair(p.hcl) &
            (p.ecl in valid_eyes) &
            bool(re.match("^\d{9}$",p.pid))
    )

In [16]:
sum([valid_passport(p) for p in passports])

121